# Objetivo

Avaliar a performance dos dois melhores modelos analisados na etapa de procura de hiperparâmetros. Na etapa anterior, os melhores modelos de RF e XGBoost são os que possuem os hiperparâmetros default.

OBSERVAÇÃO: o conjunto de dados é pequeno e para contornar o potencial viés gerado pela base de treino e validação na construção do blend, será usada a técnica Pasting. Esta técnica é parecida com o Bagging, diferenciada pela não reposição do elemento amostral escolhido. Assim, será encontrado um conjunto de estimadores blend que nos fornecerá uma resposta a respeito do ganho ou não de performance do modelo!

# Pacotes

In [ ]:
from deltalake import DeltaTable, write_deltalake
import math
import mlflow
import pickle
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, accuracy_score, average_precision_score, brier_score_loss, confusion_matrix, classification_report, ConfusionMatrixDisplay, precision_recall_curve, log_loss

# Leitura da base v1 e filtro de variáveis

Variáveis com valor acumulado de importância em ~90% e pelo valor de IV.

In [2]:
dados = DeltaTable("../1.Variaveis/tmp/dados_pp_v1").to_pandas()
dados.drop(['__index_level_0__', 'Card_Category_Gold', 'Card_Category_Platinum', 'Card_Category_Silver', 
            'Marital_Status_Married', 'Marital_Status_Single', 'Marital_Status_Unknown', 'Gender_M',
            'Dependent_count'], axis=1, inplace=True)
dados.head()

,Customer_Age,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Ct_Chng_Q4_Q1,vfm,pmcc,Income_Category_1.< 40k,Income_Category_2. >= 40k & < 60k,Income_Category_3. >= 60k & < 80k,Income_Category_4. >= 80k & < 120k,Income_Category_5. >= 120k,Education_Level_v2_1.Uneducated,Education_Level_v2_2.High School,Education_Level_v2_3.Graduate,Education_Level_v2_4.Post-Graduate,Attrition_Flag,type
0,-0.784196,1.403132,-1.337898,0.498943,0.963894,0.282975,-0.328225,-0.175537,-0.421450,1,0,0,0,0,0,0,1,0,0,Treino
1,0.720070,-0.525933,0.641818,1.408428,-0.165769,-1.527806,-0.194304,-0.208685,-1.054789,0,0,0,1,0,0,0,1,0,0,Treino
2,1.346848,-0.525933,-0.348040,0.498943,0.864865,0.894171,0.056797,-0.571459,-0.686436,0,0,0,1,0,0,0,0,1,0,Treino
3,0.218648,-0.525933,0.641818,-1.320028,-0.412731,0.369637,0.851953,0.252749,2.406712,1,0,0,0,0,0,0,1,0,0,Treino
4,-2.539173,0.117089,1.631675,1.408428,-0.858972,0.346832,-1.144306,-0.064053,-0.071911,1,0,0,0,0,1,0,0,0,0,Treino


## Separa bases

In [3]:
dados_treino = dados[dados.type == 'Treino'].drop(['type'], axis=1)
dados_val = dados[dados.type == 'Validacao'].drop(['type'], axis=1)
dados_treino_val = dados[dados.type != 'Teste'].drop(['type'], axis=1)

In [44]:
dados_treino.shape[0] + dados_val.shape[0], dados_treino_val.shape

(8607, (8607, 19))

In [4]:
X_treino_val = dados_treino_val.drop(['Attrition_Flag'], axis=1)
y_treino_val = dados_treino_val['Attrition_Flag']

X_treino = dados_treino.drop(['Attrition_Flag'], axis=1)
y_treino = dados_treino['Attrition_Flag']

X_val = dados_val.drop(['Attrition_Flag'], axis=1)
y_val = dados_val['Attrition_Flag']

In [5]:
X_treino_val_new = X_treino_val.rename(columns={'Income_Category_1.< 40k': 'Income_Category_1.40k',
                                        'Income_Category_2. >= 40k & < 60k': 'Income_Category_2.40k_60k',
                                        'Income_Category_3. >= 60k & < 80k': 'Income_Category_3.60k_80k',
                                        'Income_Category_4. >= 80k & < 120k': 'Income_Category_4.80k_120k',
                                        'Income_Category_5. >= 120k': 'Income_Category_5.120k'
                                        })

X_treino_new = X_treino.rename(columns={'Income_Category_1.< 40k': 'Income_Category_1.40k',
                                        'Income_Category_2. >= 40k & < 60k': 'Income_Category_2.40k_60k',
                                        'Income_Category_3. >= 60k & < 80k': 'Income_Category_3.60k_80k',
                                        'Income_Category_4. >= 80k & < 120k': 'Income_Category_4.80k_120k',
                                        'Income_Category_5. >= 120k': 'Income_Category_5.120k'
                                        })

X_val_new = X_val.rename(columns={'Income_Category_1.< 40k': 'Income_Category_1.40k',
                                        'Income_Category_2. >= 40k & < 60k': 'Income_Category_2.40k_60k',
                                        'Income_Category_3. >= 60k & < 80k': 'Income_Category_3.60k_80k',
                                        'Income_Category_4. >= 80k & < 120k': 'Income_Category_4.80k_120k',
                                        'Income_Category_5. >= 120k': 'Income_Category_5.120k'
                                        })

# Carrega modelos e função blend

In [7]:
def Config_Vars(X):
    # Função que configura a matriz X para ser aplicada no modelo

    # RF
    logged_model_rf = 'runs:/cfdc86fd7fa14b0d8808979bf285d839/modelo'
    rf = mlflow.sklearn.load_model(logged_model_rf)

    # Xgboost
    logged_model_xgb = 'runs:/48bade21299a413e8afc9c60dd9581e8/modelo'
    XGB = mlflow.sklearn.load_model(logged_model_xgb)

    # Aplica RF
    x_rf = rf.predict_proba(X)[:,1]

    # Aplica Xgboost
    x_xgb = XGB.predict_proba(X)[:,1]

    X_pred = pd.concat([pd.Series(x_rf).reset_index(drop=True), pd.Series(x_xgb).reset_index(drop=True)],axis=1).rename(columns={0: 'rf', 1: 'xgb'})

    return X_pred

def Classificador_Blend(X,y):

    X_pred = Config_Vars(X)

    LR = LogisticRegression()

    return LR.fit(X_pred,y)

## Exemplo de uso

As variáveis explicativas para o modelo de regressão logística são as probabilidades obtidas pelos dois modelos. E a variável resposta é a variável target original em treino e validação.

In [7]:
aux = Classificador_Blend(X_treino_new, y_treino)

In [8]:
predicao = aux.predict_proba(Config_Vars(X_treino_new))[:,1]

In [11]:
average_precision_score(y_val, aux.predict_proba(Config_Vars(X_val_new))[:,1])

np.float64(0.8445860090028201)

In [12]:
log_loss(y_val, aux.predict_proba(Config_Vars(X_val_new))[:,1], normalize=True)

0.2674357359580343

## Pasting

Diferente do exemplo anterior, aqui será usado 10 modelos, com uma amostra de 50% (bootstrap sem reposição com 50% da base de treino + validação). Para mais detalhes, usar a documentação: https://scikit-learn.org/0.15/modules/generated/sklearn.ensemble.BaggingClassifier.html.

In [8]:
X_pred = Config_Vars(X_treino_val_new)
X_pred.head()

,rf,xgb
0,0.00,0.004722
1,0.11,0.023700
2,0.00,0.017772
3,0.02,0.000051
4,0.19,0.129861


In [9]:
X_pred.shape, .5*X_pred.shape[0] # 50% da base de treino + validação

((8607, 2), 4303.5)

In [ ]:
bag_blend = BaggingClassifier(LogisticRegression(),
                              n_estimators=10,
                              max_samples=math.ceil(.5*X_pred.shape[0]),
                              bootstrap=False, # Configurando para True, trata-se do bagging
                              n_jobs=-1)
bag_blend.fit(X = X_pred, y = y_treino_val)

BaggingClassifier(bootstrap=False, estimator=LogisticRegression(),
                  max_samples=4304, n_jobs=-1)

In [76]:
average_precision_score(y_val, bag_blend.predict_proba(Config_Vars(X_val_new))[:,1])

np.float64(0.8415551875622689)

In [77]:
log_loss(y_val, bag_blend.predict_proba(Config_Vars(X_val_new))[:,1], normalize=True)

0.2242443602125264

In [81]:
bag_blend = BaggingClassifier(DecisionTreeClassifier(max_depth=5, random_state=123),
                              n_estimators=10,
                              max_samples=math.ceil(.5*X_pred.shape[0]),
                              bootstrap=False, # Configurando para True, trata-se do bagging
                              n_jobs=-1,
                              random_state=123)
bag_blend.fit(X = X_pred, y = y_treino_val)

BaggingClassifier(bootstrap=False,
                  estimator=DecisionTreeClassifier(max_depth=5,
                                                   random_state=123),
                  max_samples=4304, n_jobs=-1, random_state=123)

In [82]:
average_precision_score(y_val, bag_blend.predict_proba(Config_Vars(X_val_new))[:,1])

np.float64(0.887978067190618)

In [83]:
log_loss(y_val, bag_blend.predict_proba(Config_Vars(X_val_new))[:,1], normalize=True)

0.169319266064015

In [84]:
bag_blend.estimators_

[DecisionTreeClassifier(max_depth=5, random_state=979381764),
 DecisionTreeClassifier(max_depth=5, random_state=1540578008),
 DecisionTreeClassifier(max_depth=5, random_state=2115349175),
 DecisionTreeClassifier(max_depth=5, random_state=745079110),
 DecisionTreeClassifier(max_depth=5, random_state=268929039),
 DecisionTreeClassifier(max_depth=5, random_state=1421066145),
 DecisionTreeClassifier(max_depth=5, random_state=1770928879),
 DecisionTreeClassifier(max_depth=5, random_state=1108254685),
 DecisionTreeClassifier(max_depth=5, random_state=1151137695),
 DecisionTreeClassifier(max_depth=5, random_state=976517999)]

## Salva modelo blend

In [86]:
with open('/home/hugo/Documents/Git_GitHub/Estudo_Cartao_Credito/vCartao_Credito/2.Modelagem/bag_blender.pkl','wb') as f:
    pickle.dump(bag_blend, f)

In [87]:
with open('/home/hugo/Documents/Git_GitHub/Estudo_Cartao_Credito/vCartao_Credito/2.Modelagem/bag_blender.pkl','rb') as f:
    blend = pickle.load(f)

In [88]:
average_precision_score(y_val, blend.predict_proba(Config_Vars(X_val_new))[:,1])

np.float64(0.887978067190618)

# Conclusão

A quantidade de estimadores e o percentual amostral que será usado em cada estimador, foi definido de forma arbitrária.

O modelo blend usando regressão logística, não traz um desempenho melhor do que o XGBoost, avaliando a métrica de AUCPR e a log-loss.

O modelo blend usando a árvore de decisão, mostrou um ganho em relação ao XGBoost, com aumento do AUCPR e com redução da log-loss.

É importante relembrar que a base usada na etapa do blend é a composição das bases de treino e validação, pois a volumetria é baixa!